<a href="https://colab.research.google.com/github/saradhasarah12/BinaryPrediction-of-SmokerStatus-using-BioSignals__Kaggle/blob/main/Binary_Prediction_of_Smoker_Status_using_Bio_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=998146bd26e2ab927beac90d20c288dc7fa09387cd165c1be85237cd1c33e888
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import DecisionTreeClassifier,LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,VectorAssembler,OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pyspark.ml.tuning
Spark=SparkSession.builder.appName("Smoker").getOrCreate()

In [ ]:

train_data = Spark.read.csv("/content/train.csv", header=True, inferSchema=True)
test_data = Spark.read.csv("/content/test.csv", header=True, inferSchema=True).show()


+------+---+----------+----------+---------+--------------+---------------+-------------+--------------+--------+----------+-------------------+-----------+------------+---+---+----------+-------------+----------------+---+---+---+-------------+
|    id|age|height(cm)|weight(kg)|waist(cm)|eyesight(left)|eyesight(right)|hearing(left)|hearing(right)|systolic|relaxation|fasting blood sugar|Cholesterol|triglyceride|HDL|LDL|hemoglobin|Urine protein|serum creatinine|AST|ALT|Gtp|dental caries|
+------+---+----------+----------+---------+--------------+---------------+-------------+--------------+--------+----------+-------------------+-----------+------------+---+---+----------+-------------+----------------+---+---+---+-------------+
|159256| 40|       165|        70|     84.0|           1.2|            1.2|            1|             1|     130|        89|                107|        200|         186| 49|115|      14.2|            1|             0.9| 19| 25| 32|            0|
|159257| 80|    

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
indexer = StringIndexer(inputCol="smoking", outputCol="label").fit(train_data)
# Assemble features into a single vector
assembler = VectorAssembler(
    inputCols=[col for col in train_data.columns if col != "smoking"],
    outputCol="features"
)


In [ ]:
from pyspark.ml.classification import (LogisticRegression, RandomForestClassifier,
                                       GBTClassifier, DecisionTreeClassifier, NaiveBayes)
lr = LogisticRegression(featuresCol="features", labelCol="label")
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
gbt = GBTClassifier(featuresCol="features", labelCol="label")
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")
nb = NaiveBayes(featuresCol="features", labelCol="label")


In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, assembler, lr])
traindata, testdata = train_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
model = pipeline.fit(traindata)
predictions = model.transform(testdata)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy of Logistic Regression: {accuracy*100:.2f}%")


Accuracy of Logistic Regression: 74.65%


In [ ]:
pipeline_rf = Pipeline(stages=[indexer, assembler, rf])
model_rf = pipeline_rf.fit(traindata)
predictions_rf = model_rf.transform(testdata)
rf_accuracy = evaluator.evaluate(predictions_rf)
print(f"Accuracy of Random Forest: {rf_accuracy*100:.2f}%")


Accuracy of Random Forest: 74.89%


In [ ]:
pipeline_gbt = Pipeline(stages=[indexer, assembler, gbt])
model_gbt = pipeline_gbt.fit(traindata)
predictions_gbt = model_gbt.transform(testdata)
gbt_accuracy = evaluator.evaluate(predictions_gbt)
print(f"Accuracy of GBT: {gbt_accuracy*100:.2f}%")

Accuracy of GBT: 77.02%


In [ ]:
pipeline_dt = Pipeline(stages=[indexer, assembler, dt])
model_dt = pipeline_dt.fit(traindata)
predictions_dt = model_dt.transform(testdata)
dt_accuracy = evaluator.evaluate(predictions_dt)
print(f"Accuracy of Decision Tree: {dt_accuracy*100:.2f}%")


Accuracy of Decision Tree: 74.79%


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label")

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[smoking_indexer, assembler, rf])

In [ ]:
model = pipeline.fit(traindata)

In [ ]:
predictions = model.transform(test_data)

In [ ]:
predictions.select("age", "height(cm)", "weight(kg)","systolic","relaxation","fasting blood sugar","Cholesterol","triglyceride","HDL","LDL","hemoglobin","Urine protein","serum creatinine","AST","ALT","Gtp","dental caries", "prediction").show()

+---+----------+----------+--------+----------+-------------------+-----------+------------+---+---+----------+-------------+----------------+---+---+---+-------------+----------+
|age|height(cm)|weight(kg)|systolic|relaxation|fasting blood sugar|Cholesterol|triglyceride|HDL|LDL|hemoglobin|Urine protein|serum creatinine|AST|ALT|Gtp|dental caries|prediction|
+---+----------+----------+--------+----------+-------------------+-----------+------------+---+---+----------+-------------+----------------+---+---+---+-------------+----------+
| 20|       170|        75|     118|        75|                 79|        178|         197| 45| 93|      17.4|            1|             0.8| 27| 31| 53|            0|       1.0|
| 45|       160|        55|     150|        88|                 84|        222|         153| 69|122|      13.0|            1|             0.7| 17| 12| 16|            0|       0.0|
| 40|       165|        70|     130|        80|                104|        243|         163| 59|150|

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

crossval = CrossValidator(estimator=pipeline_rf,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=3)
cv_model = crossval.fit(traindata)
best_model = cv_model.bestModel
predictions_rf = best_model.transform(testdata)

rf_accuracy = evaluator.evaluate(predictions_rf)
print(f"Accuracy of Random Forest after hyperparameter tuning: {rf_accuracy * 100:.2f}%")


Accuracy of Random Forest after hyperparameter tuning: 74.89%


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

param_grid_gbt = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10, 15]) \
    .addGrid(gbt.maxBins, [32, 64]) \
    .addGrid(gbt.maxIter, [10, 20]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

crossval_gbt = CrossValidator(estimator=pipeline_gbt,
                              estimatorParamMaps=param_grid_gbt,
                              evaluator=evaluator,
                              numFolds=3)
cv_model_gbt = crossval_gbt.fit(traindata)

best_model_gbt = cv_model_gbt.bestModel

predictions_gbt = best_model_gbt.transform(testdata)
gbt_accuracy = evaluator.evaluate(predictions_gbt)
print(f"Accuracy of GBT after hyperparameter tuning: {gbt_accuracy * 100:.2f}%")


Accuracy of GBT after hyperparameter tuning: 76.77%


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

param_grid_dt = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.maxBins, [32, 64]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
crossval_dt = CrossValidator(estimator=pipeline_dt,
                             estimatorParamMaps=param_grid_dt,
                             evaluator=evaluator,
                             numFolds=3)

cv_model_dt = crossval_dt.fit(traindata)
best_model_dt = cv_model_dt.bestModel
predictions_dt = best_model_dt.transform(testdata)
dt_accuracy = evaluator.evaluate(predictions_dt)
print(f"Accuracy of Decision Tree after hyperparameter tuning: {dt_accuracy * 100:.2f}%")


Accuracy of Decision Tree after hyperparameter tuning: 74.86%
